## TensorFlow Core 

In [18]:
import numpy as np
import tensorflow as tf


### tensor

In [96]:
# tensorflow 就是对 tensor 计算，tensor 可以分为0阶、1阶、2阶、3阶、更高阶
# tensor 包含维度 + 数据类型
# tf.rank() 获取 tensor 阶数 

sess = tf.Session()
# 0阶：tensor 里单个值
zero_rank = tf.Variable(0,tf.int32)
print(sess.run(tf.rank(zero_rank)))

# 1 阶：类似一维数组
one_rank = tf.Variable([0,1],tf.int32)
print(sess.run(tf.rank(one_rank)))

# 2 阶：类似二维数组
two_rank = tf.Variable([[0,1],[2,3]],tf.int32)
print(sess.run(tf.rank(two_rank)))

# 3 阶：类似三维数组
three_rank = tf.random.uniform(((2,3,2)),1,2)
print(sess.run((tf.rank(three_rank),three_rank)) )

0
1
2
(3, array([[[1.3162619, 1.0168624],
        [1.6151062, 1.8095675],
        [1.7547767, 1.8808796]],

       [[1.6135429, 1.8138312],
        [1.8902813, 1.7622585],
        [1.8953396, 1.5058954]]], dtype=float32))


#### 形状 

In [44]:
# 获取 tensor 形状，shape()
# three_rank.shape = [2,3,2]
zeros = tf.zeros(three_rank.shape[1])
ones = tf.ones(three_rank.shape)
print(sess.run((zeros,ones)))

# 改变形状
rank_three_tensor = tf.ones([3, 4, 5])
# 在 shape 中，-1 代表自动补全
change_rank = tf.reshape(rank_three_tensor,[4,3,-1])
# -1 代表补全，但如果不整除则会报错；3*4*5 / 26 = 2..3333
error_rank = tf.reshape(rank_three_tensor,[13,2,-1])
print(sess.run((change_rank,error_rank)))

(array([0., 0., 0.], dtype=float32), array([[[1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.]]], dtype=float32))


#### 数据类型 

In [48]:
# 一个 tensor 只能有一种数据类型

int_type = tf.constant([1,2,3],tf.int32)
# tf.cast() 转换类型
float_type = tf.cast(int_type,dtype=tf.float32)

print(int_type.dtype)
print(float_type.dtype)

<dtype: 'int32'>
<dtype: 'float32'>


#### 评估张量 

In [73]:
# 临时输出值 eval()

constant = tf.constant([1,2,3])
sum_val = constant + constant
# 不需要 run 可临时评估；但有占位符的情况下，需要提供值才能计算临时值
sum_val.eval(session=sess)

<dtype: 'int32'>


#### 输出张量

In [97]:
# 打印张量 调试使用
t = tf.constant(1)
t = tf.Print(sum_val,[sum_val],"s")
sess.run(t)

array([2, 4, 6])

## Graph

In [56]:
a = tf.constant(4.0,tf.float32)
b = tf.constant(8.0,tf.float32)
c = a + b
# 此时不是输出值，而是 tensor 状态
print(a)
print(b)
print(c)

Tensor("Const_10:0", shape=(), dtype=float32)
Tensor("Const_11:0", shape=(), dtype=float32)
Tensor("add_11:0", shape=(), dtype=float32)


### 命名 

In [95]:
# 以下关于命名不涉及变量的命名规则

# 当命名被占用，会在命名后加 "_index"
c_0 = tf.constant(0, name="c")  # => operation named "c"

# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # => operation named "c_1"

# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")  # => operation named "outer/c"

  # Name scopes nest like paths in a hierarchical file system.
with tf.name_scope("inner"):
    c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"

  # Exiting a name scope context will return to the previous prefix.
    c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"

  # Already-used name scopes will be "uniquified".
    with tf.name_scope("inner"):
        c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"
# 
x = tf.Variable([1,2],dtype=tf.float16)
x_ops = x.assign_add([10,2],name = "ops")
# ops:0 => 操作的名称:该张量的索引
print(x_ops.name)

ops:0


### 不同设备 

In [ ]:
# 多设备 tensorflow 稍后看

### 类似于张量的对象 

### tensorBoard

In [3]:
# 保存 graph ，用 tensorboard 打开

tf_write = tf.summary.FileWriter('.')
tf_write.add_graph(tf.get_default_graph())
# 在当前文件下生成 “events.out.tfevents.{timestamp}.{hostname}” 文件
# 一：tensorboard --logdir dirpath 加载保存的 graph 文件
# 二：localhost:6066 在浏览器展示 tensorboard

### Session 

In [ ]:
graph：存储要运算的 tensor 流程
Session：计算 graph

In [4]:
sess = tf.Session()
# 此时才是输出 c 值 12.0；
print(sess.run(c))

12.0


In [5]:
# 同理计算
print(sess.run({'ab':(a,b),'c':c}))

{'ab': (4.0, 8.0), 'c': 12.0}


In [8]:
vec = tf.random.uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
# 每次 run 都会重新计算 tensor 的值，所有第一次和第二次输出的值不是相差1
# 但同个 run 下，tensor 值共用，故第三次输出 out1 和 out2 相差1
print(sess.run(out1))
print(sess.run(out2))
print(sess.run((out1,out2)))

[1.7616307 1.9310986 1.3928301]
[2.4288657 2.9211712 2.2150261]
(array([1.3132862, 1.3063877, 1.5273491], dtype=float32), array([2.3132863, 2.3063877, 2.527349 ], dtype=float32))


### 占位符

In [12]:
# 占位符类似函数参数，只提供参数类型，到真正执行的时候才提供真实值

x = tf.placeholder(tf.int32)
y = tf.placeholder(tf.int32)
z = x + y

# 占位符在 run 时必须用 feed_dict 提供真实值，否则会报错
print(sess.run(z,feed_dict={x:3,y:5}))
print(sess.run(z,feed_dict={x:[1,3],y:[5,9]}))

8
[ 6 12]


### 数据集

In [16]:
# 将数据集转化为 tf.data.Iterator，然后用 get_next 逐个取出
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]

# my_data 数据集先转化为 iterator
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

# get_next() 超出范围后，爆 OutOfRangeError异常
while(True):
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break


[0 1]
[2 3]
[4 5]
[6 7]


### 层 

In [12]:
import tensorflow as tf

x = tf.placeholder(tf.float32,shape=[None,3])
# 创建 Dense 层;units 输出是几个值
linear_mode = tf.layers.Dense(units=1)
y = linear_mode(x)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(y,feed_dict={x:[[1,2,3],[2,3,4]]}))

[[4.1774855]
 [6.1031528]]


### 特征列 

In [22]:
# 特征数据
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

# 标签特征值需要 indicator_column 转化
department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])

department_column = tf.feature_column.indicator_column(department_column)

# 组合特征的列
columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

# 转化特征
inputs = tf.feature_column.input_layer(features, columns)

with tf.Session() as sess:
    sess.run(init)
    # 特征列内部有对照表 也需要初始化
    sess.run(tf.tables_initializer())
    # 第一、二列是 one hot 值，第三列是 sales
    print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


### 训练 

In [35]:
# 数据准备
x = tf.constant([[1],[2],[3],[4]],dtype=tf.float32)
y_true = tf.constant([[2],[5],[7],[1]],dtype=tf.float32)

# 选择模型
linear_mode = tf.layers.Dense(units=1)

# 预测
y_pred = linear_mode(x)

# 定义损失函数
loss = tf.losses.mean_squared_error(labels=y_true,predictions=y_pred)

# 训练 - 寻找最小损失值 - 梯度下降
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# tensorflow 相关操作
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        _,loss_val = sess.run((train,loss))
        print(loss_val)
    print(sess.run(y_pred))

16.270452
15.0604315
14.220334
13.636921
13.23162
12.949907
12.753956
12.617515
12.522367
12.455876
12.409275
12.376472
12.353249
12.3366785
12.324723
12.315975
12.309455
12.304483
12.300587
12.297442
12.294819
12.2925625
12.290563
12.288742
12.28705
12.285448
12.283911
12.282425
12.280973
12.279552
12.27815
12.276765
12.275394
12.274039
12.272691
12.271356
12.270029
12.268711
12.267401
12.266101
12.26481
12.263522
12.262245
12.260977
12.259716
12.258461
12.2572155
12.255975
12.254744
12.253519
12.252303
12.251095
12.249891
12.248695
12.247507
12.246326
12.245152
12.243986
12.242827
12.241674
12.240528
12.23939
12.238256
12.23713
12.236012
12.2348995
12.233794
12.232695
12.231604
12.230518
12.229439
12.228365
12.227298
12.226238
12.225186
12.224137
12.223098
12.222062
12.221034
12.220009
12.218995
12.217985
12.216979
12.215981
12.21499
12.214003
12.213021
12.212047
12.211079
12.210115
12.209158
12.208207
12.20726
12.20632
12.205385
12.204456
12.203532
12.202613
12.201702
12.200794
[[1.

## 变量 

In [46]:
# 指定名称 my_var, 值 tf.constant([32,42])，类型根据值 自动化
my_var = tf.get_variable("my_var",initializer=tf.constant([32,42]))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(my_var))

[32 42]


### 变量集合 

In [52]:
# 变量默认是在 tf.GraphKeys.GLOBAL_VARIABLES 集合，但可以手动修改
# my_var 存放在本地 
my_var1 = tf.get_variable("my_var2",initializer=tf.constant([10,42]),collections=[tf.GraphKeys.LOCAL_VARIABLES])

# 放置到自己的集合 "my_collection"；自定义变量集合无需显式创建，添加后会自动创建
tf.add_to_collection("my_collection",my_var1)

# 变量集合包含那些变量
tf.get_collection("my_collection")

[<tf.Variable 'my_var1:0' shape=(2,) dtype=int32_ref>,
 <tf.Variable 'my_var2:0' shape=(2,) dtype=int32_ref>]

### 初始化变量 

In [61]:
# global_variables_initializer 只初始化 在tf.GraphKeys.GLOBAL_VARIABLES 中的变量
sess = tf.Session()
sess.run(tf.global_variables_initializer())

y = 1+my_var
x = 1+my_var1
print(sess.run(y))
# 显然这个会报错，因为 my_var1 在 "my_collection" 变量集合，没有被初始化
# print(sess.run(x))

# 手动初始化单个变量
sess.run(my_var1.initializer)
# 这里才不会报错
print(sess.run(x))

# 查询那些变了没初始化
print(sess.run(tf.report_uninitialized_variables()))

# global_variables_initializer 初始化变量时，没有固定的顺序
# 如果变量的初始化需要其他变量时，应使用 variable.initialized_value() ，防止其他变量没初始化
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
f = tf.get_variable("f",shape=(),initializer=v.initialized_value() + 1)

[33 43]
[11 43]
[b'my_var1']


### 使用变量 

In [73]:
# assign 操作符,将 assign 中计算的值赋予变量

v = tf.get_variable("v4",shape=(),initializer=tf.zeros_initializer())
# v_ops 是操作 不是变量
v_ops = v.assign(v + 1)

v_add = v.assign_add(10)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(v_ops)
    sess.run(v_ops)
    sess.run(v_ops)
    sess.run(v_add)
    # 读取值
    print(v.eval())

13.0


### 共享变量 

In [80]:
# 显式传递 tf.Variable 对象
# 将 tf.Variable 对象隐式封装在 tf.variable_scope 对象内。

def cov_relu(inputs,weight_shape,bias_shape):
    weights = tf.get_variable("weights2",shape=weight_shape,initializer=tf.random_uniform_initializer())
    bases = tf.get_variable("base2",shape=bias_shape,initializer=tf.constant_initializer(0.0))
    # 4维 卷积计算成2维
    cov = tf.nn.conv2d(inputs,weights,strides=[1, 1, 1, 1],padding='SAME')
    return tf.nn.relu(cov + bases)

input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])
# 以下代码会出错，在cov_relu 函数中，"weights1" 使用get_variable 获取，
# 占用了此变量名，下次无法使用，有2种方法来解决：增加变量命名空间；在相同变量命名空间下重复使用
# x = cov_relu(x, weight_shape=[5, 5, 32, 32], bias_shape = [32]) 

#### 变量作用域 

In [81]:
# variable_scope 增加变量域
with tf.variable_scope("cov1"):
    x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])
with tf.variable_scope("cov2"):
    x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])

In [87]:
# 在同变量域下重新使用相同变量名，reuse=True
with tf.variable_scope("model10"):
    x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])
with tf.variable_scope("model10",reuse=True):
    x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])

# 更简单的方法，不传作用域名称，传作用域
with tf.variable_scope("model_scope") as model_scope:
    x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])
with tf.variable_scope(model_scope,reuse=True):
    x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])

In [86]:
# 重用变量名的另一种方法 reuse_variables: 重用该域下所有变量
with tf.variable_scope("model2") as scope:
    x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])
    scope.reuse_variables()
    x = cov_relu(input1, weight_shape=[5, 5, 32, 32], bias_shape=[32])